In [1]:
import pandas
%load_ext autoreload

%autoreload 2



In [2]:
from mlflow import MlflowClient
import mlflow
import numpy as np
import pandas as pd
from functools import reduce
import matplotlib.pyplot as plt
import collections
import matplotlib as mpl
import numpy as np
from collections import defaultdict
import scienceplots
from sklearn import datasets
from sklearn.datasets import make_blobs
from cycler import cycler
from pathlib import Path

plt.style.use(['science', 'bright'])


In [3]:
client = MlflowClient(tracking_uri="http://192.168.1.181:5010")
mlflow.set_tracking_uri("http://192.168.1.181:5010")


In [4]:
complexities_df = pd.read_csv('complexities.csv')\
    .rename(columns={"shuffle": "params.data_shuffle_random_state", "dataset_name": "params.dataset", "should_take_test": "params.should_take_test"})\
    .astype({"params.data_shuffle_random_state": str, "params.should_take_test": str, "params.dataset": str})

In [5]:
dataset_details_df = pd.read_csv('dataset-details.csv')

In [6]:
dataset_details_df

,name,features_count,classes_count,size,integer,real,category
0,appendicitis,7,2,106,0,7,0
1,australian,14,2,690,5,3,6
2,bands,19,2,365,6,13,0
3,breast,9,2,277,0,0,9
4,bupa,6,2,345,5,1,0
5,crx,15,2,653,3,3,9
6,haberman,3,2,306,3,0,0
7,heart,13,2,270,12,1,0
8,hepatitis,19,2,80,17,2,0
9,housevotes,16,2,232,0,0,16


In [7]:
complexities_df

,params.dataset,params.data_shuffle_random_state,params.should_take_test,f1,f1-time,f2,f2-time,f3,f3-time,f4,...,t4,t4-time,clsCoef,clsCoef-time,hubs,hubs-time,density,density-time,lsc,lsc-time
0,ionosphere,42,True,0.646397,0.000333,0.0,0.000181,0.789773,0.000392,0.000000,...,0.666667,0.001177,0.254879,0.034276,0.462905,0.016014,0.676299,0.012088,0.942794,0.001180
1,ionosphere,42,False,0.542857,0.000266,0.0,0.000185,0.828571,0.000379,0.000000,...,0.666667,0.001489,0.277978,0.030755,0.502582,0.015274,0.725714,0.011925,0.950237,0.008652
2,ionosphere,69,True,0.626075,0.000249,0.0,0.000169,0.795455,0.000361,0.022727,...,0.666667,0.001286,0.262925,0.032821,0.482017,0.015486,0.696818,0.011984,0.931495,0.001933
3,ionosphere,69,False,0.651531,0.000252,0.0,0.000176,0.817143,0.000388,0.000000,...,0.666667,0.001110,0.320438,0.030336,0.505417,0.016369,0.733136,0.012127,0.956278,0.001789
4,ionosphere,22,True,0.568574,0.000221,0.0,0.000168,0.795455,0.000382,0.000000,...,0.666667,0.001399,0.268297,0.032331,0.486328,0.016573,0.707662,0.015982,0.952738,0.001446
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,hepatitis,22,False,0.554017,0.000173,0.0,0.000128,0.425000,0.000224,0.000000,...,0.210526,0.000646,0.860179,0.003084,0.821947,0.002781,0.960256,0.002215,0.895625,0.000364
286,hepatitis,1,True,0.630137,0.000179,0.0,0.000134,0.250000,0.000230,0.000000,...,0.210526,0.000638,0.789286,0.003011,0.844122,0.002828,0.962821,0.002256,0.821250,0.000373
287,hepatitis,1,False,0.548920,0.000174,0.0,0.000129,0.400000,0.000223,0.000000,...,0.210526,0.000610,0.870179,0.002985,0.823346,0.002673,0.964103,0.002257,0.885000,0.000368
288,hepatitis,99,True,0.463576,0.000179,0.0,0.000133,0.325000,0.000216,0.000000,...,0.263158,0.000631,0.803214,0.003004,0.822523,0.002675,0.964103,0.002289,0.859375,0.000378


In [8]:
COMPLEXITY_METRICS = [m for m in complexities_df.columns.tolist()[3:] if 'time' not in m]

In [9]:
complexities_df = pd.merge(complexities_df, dataset_details_df, left_on='params.dataset', right_on='name', how='left').drop('name', axis=1).drop('classes_count', axis=1)

In [10]:
def cached(runs_name, directory="base-runs-cache"): 
    csv_path = Path(f"./{directory}/{runs_name}.pickle")
    return csv_path.is_file()
def load_cache(runs_name, directory="base-runs-cache"):
    return pd.read_pickle(f"./{directory}/{runs_name}.pickle")
    # .astype({"params.data_shuffle_random_state": str, "params.should_take_test": str, "params.dataset": str})
def cache(runs, runs_name, directory="base-runs-cache"):
    runs.to_pickle(f"./{directory}/{runs_name}.pickle")

In [11]:
def get_runs_for(name, client = client):
    return mlflow.search_runs(client.get_experiment_by_name(name).experiment_id)

def get_metrics(artifact_uri, name="cls_report"):
    results = mlflow.artifacts.load_dict(f"{artifact_uri}/{name}")
    
    return results

In [12]:
CLASSIFICATION_METRICS = ['cls_report', 'imb_cls_report']
DATASET_PARAMS = ['params.should_take_test', 'params.dataset', 'params.data_shuffle_random_state']
RF_METRICS = ['metrics.depth_mean',
              'metrics.depth_min',
              'metrics.depth_median',
              'metrics.n_leaves_max',
              'metrics.n_leaves_min',
              'metrics.n_leaves_mean',
              'metrics.depth_max',
              'metrics.n_leaves_median',]


In [13]:
classification_metrics_extractors = {
    "test_index_balanced_accuracy": lambda row: row['imb_cls_report'].apply(lambda it: it['avg_iba']),
    "test_geometric_mean": lambda row: row['imb_cls_report'].apply(lambda it: it['avg_geo']),
    "test_f1": lambda row: row['imb_cls_report'].apply(lambda it: it['avg_f1']),
    "test_specificity": lambda row: row['imb_cls_report'].apply(lambda it: it['avg_spe']),
    "test_recall": lambda row: row['imb_cls_report'].apply(lambda it: it['avg_rec']),
    "test_precision": lambda row: row['imb_cls_report'].apply(lambda it: it['avg_pre']),
    "test_balanced_accuracy": lambda row: row['cls_report'].apply(lambda it: it['macro avg']['recall']),
    "test_accuracy": lambda row: row['cls_report'].apply(lambda it: it['accuracy'])
}

def calculate_rf_compexity(complex_dt):
    return sum([int(n_leaves) for n_leaves, depth in zip(complex_dt['dts']['n_leaves'].values(), complex_dt['dts']['depth'].values())])

In [14]:
if cached("base_runs_dt"):
    base_runs_dt = load_cache("base_runs_dt")
else:
    base_runs_dt = get_runs_for("base_runs_dt")\
        .assign(cls_report=lambda row: row['artifact_uri'].apply(lambda v: get_metrics(v, "cls_report.json")))\
        .assign(imb_cls_report=lambda row: row['artifact_uri'].apply(lambda v: get_metrics(v, "imb_cls_report.json")))\
        [CLASSIFICATION_METRICS + DATASET_PARAMS + ['metrics.depth', "metrics.n_leaves"]]\
        .assign(**classification_metrics_extractors)\
        .assign(complexity_dt=lambda row: row['metrics.n_leaves'])

In [15]:
cache(base_runs_dt, "base_runs_dt")

In [16]:
if cached("base_runs_rf"):
    base_runs_rf = load_cache("base_runs_rf")
else:
    base_runs_rf = get_runs_for("base_runs_rf")\
        .assign(cls_report=lambda row: row['artifact_uri'].apply(lambda v: get_metrics(v, "cls_report.json")))\
        .assign(imb_cls_report=lambda row: row['artifact_uri'].apply(lambda v: get_metrics(v, "imb_cls_report.json")))\
        .assign(complex_dt=lambda row: row['artifact_uri'].apply(lambda v: get_metrics(v, "complex_dt.json")))\
        [CLASSIFICATION_METRICS + DATASET_PARAMS + RF_METRICS + ["complex_dt"]]\
        .assign(**classification_metrics_extractors)\
        .assign(complexity_rf=lambda row: row['complex_dt'].apply(calculate_rf_compexity))

In [17]:
cache(base_runs_rf, "base_runs_rf")

In [18]:
if cached("base_runs_rf_extended"):
    base_runs_rf_extended = load_cache("base_runs_rf_extended")
else:
    base_runs_rf_extended = get_runs_for("base_runs_rf_extended")\
        .assign(cls_report=lambda row: row['artifact_uri'].apply(lambda v: get_metrics(v, "cls_report.json")))\
        .assign(imb_cls_report=lambda row: row['artifact_uri'].apply(lambda v: get_metrics(v, "imb_cls_report.json")))\
        .assign(complex_dt=lambda row: row['artifact_uri'].apply(lambda v: get_metrics(v, "complex_dt.json")))\
        [CLASSIFICATION_METRICS + DATASET_PARAMS + RF_METRICS + ["complex_dt"]]\
        .assign(**classification_metrics_extractors)\
        .assign(complexity_rf=lambda row: row['complex_dt'].apply(calculate_rf_compexity))

In [19]:
cache(base_runs_rf_extended, "base_runs_rf_extended")

In [20]:
def calculate_complexity_oner_greedy(row):
    rules = row['rules']

    return len(rules)

In [21]:
if cached("base_runs_oner"):
    base_runs_oner = load_cache("base_runs_oner")
else:
    base_runs_oner = get_runs_for("base_runs_oner")\
        .assign(cls_report=lambda row: row['artifact_uri'].apply(lambda v: get_metrics(v, "cls_report.json")))\
        .assign(imb_cls_report=lambda row: row['artifact_uri'].apply(lambda v: get_metrics(v, "imb_cls_report.json")))\
        .assign(rules=lambda row: row['artifact_uri'].apply(lambda v: get_metrics(v, "rules.json")))\
        [CLASSIFICATION_METRICS + DATASET_PARAMS + ['metrics.rules_no', 'metrics.depth', 'rules']]\
        .assign(**classification_metrics_extractors)\
        .assign(complexity_oner=lambda row: row['rules'].apply(calculate_complexity_oner_greedy))

In [22]:
cache(base_runs_oner, "base_runs_oner")

In [23]:
def calculate_complexity_rulefit(row):
    rules = row['rules']

    return len(rules)

In [24]:
if cached("base_runs_rulefit"):
    base_runs_rulefit = load_cache("base_runs_rulefit")
else:
    base_runs_rulefit = get_runs_for("base_runs_rulefit")\
        .assign(cls_report=lambda row: row['artifact_uri'].apply(lambda v: get_metrics(v, "cls_report.json")))\
        .assign(imb_cls_report=lambda row: row['artifact_uri'].apply(lambda v: get_metrics(v, "imb_cls_report.json")))\
        .assign(rules=lambda row: row['artifact_uri'].apply(lambda v: get_metrics(v, "rules.json")))\
        [CLASSIFICATION_METRICS + DATASET_PARAMS + ['metrics.max_rules', 'metrics.rules_no', 'metrics.terms_no', 'rules']]\
        .assign(**classification_metrics_extractors)\
        .assign(complexity_rulefit=lambda row: row['rules'].apply(calculate_complexity_rulefit))


In [25]:
cache(base_runs_rulefit, "base_runs_rulefit")

In [26]:
if cached("base_runs_greedy"):
    base_runs_greedy = load_cache("base_runs_greedy")
else:
    base_runs_greedy = get_runs_for("base_runs_greedy")\
        .assign(cls_report=lambda row: row['artifact_uri'].apply(lambda v: get_metrics(v, "cls_report.json")))\
        .assign(imb_cls_report=lambda row: row['artifact_uri'].apply(lambda v: get_metrics(v, "imb_cls_report.json")))\
        .assign(rules=lambda row: row['artifact_uri'].apply(lambda v: get_metrics(v, "rules.json")))\
        [CLASSIFICATION_METRICS + DATASET_PARAMS + ['metrics.depth', 'metrics.rules_no', 'rules']]\
        .assign(**classification_metrics_extractors)\
        .assign(complexity_greedy=lambda row: row['rules'].apply(calculate_complexity_oner_greedy))


In [27]:
cache(base_runs_greedy, "base_runs_greedy")